# Prerequisites and Installation

## Virtual Environment

Ensure to have PIP installed. On Ubuntu:

```sh
apt install python3-pip
```

Create your virtual environment

```sh
python3 -m venv ./venv
```

Then once it's created, you need to initialise it before setting the Jupyter
Kernel:

```sh
. ./venv/bin/activate
```

Choose the `venv` Python kernel from within VSCode. Follow instructions for
installing the `ipykernel` which is needed to run Jupiter. It's installed safely
in your virtual environment.

If you can't see the `venv` as the kernel in VSCode, try changing to the
directory where the `venv` folder is (not the `venv` folder itself, but one
before it) and starting VSCode from there. Else you may unintentionally install
all the packages needed in your global or user configuration.

## Ubuntu Packages

Install the following packages.

- Allows exporting Jupyter notebooks as PDF:

  ```sh
  apt install texlive-xetex
  ```

In [ ]:
%pip install matplotlib

## Python Tools

Install the `matplotlib`, which will also install `numpy`.

# Introduction

This document covers interpreting the output of the program `udp_load` to plot
the data over multiple instances for throughput and CPU load.

In [ ]:
CONFIG_FILE = "data/rpi4_rpios_sendto_eth.txt"

MODE="Linux"
MODE="QNX"

Read the `CONFIG_FILE` into memory:

In [ ]:
import os
import re

if not os.path.isfile(CONFIG_FILE):
    raise Exception(f"File '{CONFIG_FILE}' not found")

FIRSTLINE="^UDP Talker Parameters"
PARAMTHREADS="Threads: (\d+)"
PARAMPKTSEC="Packets/sec: (\d+)"
PARAMTOTCPU="Total CPU Busy: (\d+\.\d+)\%"
PARAMPRCCPU="Process CPU Busy: (\d+\.\d+)\%"
PARAMSENT="Packets Sent: (\d+)"
PARAMEXP="Packets Expected: (\d+)"

# Dataformat (all dictionaries)
#  data[threads][pkt/sec]{field}

data = {}
def insert_data_line(thread, pktsec, line):
    global data
    if thread == 0:
        return
    if pktsec == 0:
        return
    if threads not in data:
        data[threads] = {}
    if pktsec not in data[threads]:
        data[threads][pktsec] = None
    data[threads][pktsec] = line

threads = 0
pktsec = 0
dataline = {}
with open(CONFIG_FILE, encoding="utf-8") as config_file:
    for line in config_file:
        match = re.match(FIRSTLINE, line.strip())
        if match:
            insert_data_line(threads, pktsec, dataline)
            threads = 0
            pktsec = 0
            dataline = {}
            continue
        match = re.match(PARAMTHREADS, line.strip())
        if match:
            threads = int(match.group(1))
            continue
        match = re.match(PARAMPKTSEC, line.strip())
        if match:
            pktsec = int(match.group(1))
            continue
        match = re.match(PARAMTOTCPU, line.strip())
        if match:
            dataline["TotalCpu"] = float(match.group(1))
            continue
        match = re.match(PARAMPRCCPU, line.strip())
        if match:
            dataline["ProcessCpu"] = float(match.group(1))
            continue
        match = re.match(PARAMSENT, line.strip())
        if match:
            dataline["PktSent"] = int(match.group(1))
            # Bandwidth in Mbps. Enusre in the tests that the default -d30000
            # (30s) is used.
            dataline["PktBW"] = int(match.group(1)) / 30 * 1538 * 8 / 1000000
            continue
        match = re.match(PARAMEXP, line.strip())
        if match:
            dataline["PktExpected"] = int(match.group(1))
            continue
    insert_data_line(threads, pktsec, dataline)


Plot the data. The height of the bandwidth is assuming the input files were
recorded for 30s duration (`-d30000`) and a payload of 1472 bytes (`-s1472`)
(total packets sent is divided by 30s with each packet of 1538 bytes with
overhead).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def get_data(field):
    global data

    # Get our X and Y values. The Z-Values are then what we want to plot.
    t = sorted(data.keys())
    p = []
    for it in t:
        for ip in data[it].keys():
            if ip not in p:
                p.append(ip)
        p = sorted(p)
    Y, X = np.meshgrid(p, t)
    Z = np.zeros((len(t), len(p)))
    for it in range(len(t)):
        for ip in range(len(p)):
            Z[it][ip] = data[t[it]][p[ip]][field]
    return X, Y, Z

def get_plot(fig, ax, field, cmap):
    global data

    if ax is None:
        ax = fig.add_subplot(projection="3d")
    ax.set_ylabel('Target Packets/Sec')
    ax.set_xlabel('Threads')
    X, Y, Z = get_data(field)
    ax.plot_surface(X, Y, Z, edgecolor="black", cmap=cmap, alpha=0.7, linewidth=0.5)
    #ax.set_xticks(data.keys())
    ax.set_xticks([1, 2, 3, 4])
    #ax.set_yticks(Y[0])
    ax.tick_params(axis='both', which='major', pad=0, grid_linewidth=0.5)
    ax.tick_params(axis='y', which='major', labelright=True, labelbottom=False, labeltop=False, labelleft=False)
    return ax

#plt.rcParams.update({'font.size': 3})
fig = plt.figure(facecolor="lightgrey", figsize=(20,10), dpi=150)
ax1 = fig.add_subplot(121, projection="3d")

if MODE == "Linux":
    get_plot(fig, ax1, "ProcessCpu", "coolwarm")
elif MODE == "QNX":
    get_plot(fig, ax1, "TotalCpu", "coolwarm")
    get_plot(fig, ax1, "ProcessCpu", "coolwarm")
ax1.set_title(f"Plot of CPU ({CONFIG_FILE})")
ax1.set_box_aspect(aspect=None, zoom=0.9)
ax1.set_zlabel('CPU (%)')

ax2 = fig.add_subplot(122, projection="3d")
get_plot(fig, ax2, "PktBW", "viridis")
ax2.set_title(f"Total Packets Sent ({CONFIG_FILE})")
ax2.set_box_aspect(aspect=None, zoom=0.9)
ax2.set_zlabel('Bandwidth (Mbps)')
